In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder 

In [ ]:
train_data = pd.read_csv("train.csv").drop(columns=['Date'])
test_data = pd.read_csv('test.csv').drop(columns=['Date'])
#cat_cols
train_data = pd.get_dummies(train_data)
test_data = pd.get_dummies(test_data)
#cat_cols = train_data.keys()

In [ ]:
len(test_data.keys())

64

In [ ]:
len(train_data.keys())

65

In [ ]:
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
#encoder = OrdinalEncoder()

In [ ]:
#KNN find missing value
tmp = imputer.fit_transform(train_data)

In [ ]:
#imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
tmp2 = imputer.fit_transform(test_data)

In [ ]:
#tmp2 = imputer.fit_transform(test_data)

In [ ]:
#a = pd.DataFrame(tmp)
encode_data = pd.DataFrame(tmp)
encode_test_data = pd.DataFrame(tmp2)
#tmp2['Weather'] = np.zeros((len(test_data),))
#encode_data = tmp
#encode_test_data = tmp2
#test_data.keys()

In [ ]:
#encode_data

In [ ]:
len(encode_test_data.keys())

113

In [ ]:
'''def encode(data):
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data

#create a for loop to iterate through each column in the data
for columns in cat_cols:
    encode(train_data[columns])
encode_data = pd.DataFrame(np.round(imputer.fit_transform(train_data)),columns = train_data.columns)

In [ ]:
'''#test_data['Weather'] = np.zeros((len(test_data),))
for columns in cat_cols:
    if columns == 'Weather':continue
    encode(test_data[columns])
encode_test_data = pd.DataFrame(np.round(imputer.fit_transform(test_data)),columns = test_data.columns)
encode_test_data['Weather'] = np.zeros((len(test_data),))

In [ ]:
#train_data

In [ ]:
#encode_data = pd.DataFrame(np.round(imputer.fit_transform(train_data)),columns = train_data.columns)

In [ ]:
#encode_data
encode_data.to_csv(r'encoded_df.csv', index = True)

In [ ]:
#encode_test_data = pd.DataFrame(np.round(imputer.fit_transform(test_data)),columns = test_data.columns)
encode_test_data.to_csv(r'encoded_test_df.csv', index = True)

In [ ]:
#splitting data
'''from sklearn.model_selection import train_test_split

train_end_index = len(encode_data)
encode_data_concat = pd.concat([encode_data,encode_test_data],sort=False)

x_train, x_val, y_train, y_val = train_test_split(
    encode_data_concat.drop(columns=['Weather']).values[:train_end_index,:],
    encode_data_concat['Weather'].values[:train_end_index],
    test_size = 0.5,
    shuffle = True
)

x_test = encode_data_concat.drop(columns=['Weather']).values[train_end_index:,:]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(
    encode_data.drop(columns=[15]).values,
    encode_data[15].values,
    test_size = 0.1,
    #shuffle = True
)
x_test = encode_test_data.values
#x_train = encode_data.drop(columns=[15]).values
#y_train = encode_data[15].values
#dealing imbalance
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 42)
x_train,y_train = sm.fit_resample(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#encode_test_data
pd.DataFrame(y_train)

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
56661,1.0
56662,1.0
56663,1.0
56664,1.0


In [ ]:
#decision
from sklearn.tree import DecisionTreeClassifier
DecisionTreeClassifier_model = DecisionTreeClassifier()
DecisionTreeClassifier_model.fit(x_train,y_train)
model = DecisionTreeClassifier_model 

In [ ]:
#random forest
from sklearn.ensemble import RandomForestClassifier
RandomForestClassifier_model = RandomForestClassifier(verbose=2)
RandomForestClassifier_model.fit(x_train,y_train)
model = RandomForestClassifier_model

In [ ]:
#KNC
from sklearn.neighbors import KNeighborsClassifier
KNeighborsClassifier_model = KNeighborsClassifier(n_neighbors=5)
KNeighborsClassifier_model.fit(x_train,y_train)
model = KNeighborsClassifier_model

In [ ]:
#SVM
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
SVC_model = SVC(gamma='auto',verbose=2)
SVC_model.fit(x_train,y_train)
model = SVC_model

[LibSVM]

In [ ]:
from lightgbm import LGBMClassifier
LGBMClassifier_model = LGBMClassifier(objective='binary')
LGBMClassifier_model.fit(x_train,y_train)
model = LGBMClassifier_model  

In [ ]:
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
SGD_model = make_pipeline(StandardScaler(),SGDClassifier(max_iter=1000, tol=1e-3,alpha=0.01))
SGD_model.fit(x_train, y_train)
model = SGD_model


In [ ]:
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
from xgboost.sklearn import XGBClassifier
XGB_model = XGBClassifier(learning_rate=0.1,reg_alpha=1,n_estimators=100)
XGB_model.fit(x_train,y_train,eval_metric='auc',verbose=2)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=1, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split
model = LGBMClassifier_model
#for validation data
print("Validation data: ")
y_pred = model.predict(x_val)
print("Accuaracy: %f" % accuracy_score(y_val, y_pred))
print("F1-score: %f" % f1_score(y_val, y_pred))
print("Training data: ")
y_pred = model.predict(x_train)
print("Accuaracy: %f" % accuracy_score(y_train, y_pred))
print("F1-score: %f" % f1_score(y_train, y_pred))
print("====================")

#for testing data
#print("Testing data:")
#y_test = pd.read_csv("output_df.csv")
#y_test = y_test['Weather'].values

y_pred = model.predict(x_test)
#print("Accuaracy: %f" % accuracy_score(y_test, y_pred))
#print("F1-score: %f" % f1_score(y_test, y_pred))

#same_n = 0
#for i,j in zip(y_pred,y_test):
 # if i==j: same_n+=1
#print("my acc: ", same_n/len(y_pred))

#output sunbmission
list_id = list(range(len(y_pred)))
df = {'Id':list_id, 'Weather': list(map(bool,y_pred))}
output_df = pd.DataFrame(data=df)
output_df.to_csv(r'output_df_xgb.csv', index = False)

Validation data: 
Accuaracy: 0.903299
F1-score: 0.388489
Training data: 
Accuaracy: 0.949674
F1-score: 0.948045
